In [1]:
# ce code fonctionne. si on veut avoir les fichiers des autres pages sans teiheader, merci de le modifier

import json
import os
import re
import lxml.etree as ET
from lxml import etree
import dateparser
import datetime

# Les fonctions

tag_counts = {}

def generate_id(tag):
    """Génère un identifiant unique pour une balise en fonction de son type."""
    if tag not in tag_counts:
        tag_counts[tag] = 1
    else:
        tag_counts[tag] += 1
    return f"{tag}{tag_counts[tag]}"

 # Ajout des éléments de la TeiHeader ________________________________________________________________________________

def create_teiheader(data):
    
    #for i in range(len(data)):
        
        #if "comment" in data[i]:

# fileDesc______________________________________________________________________

    fileDesc = ET.SubElement(teiHeader , "fileDesc")

    # Titre______________________________________________________________________
    
    global titleStmt
    titleStmt = ET.SubElement(fileDesc , "titleStmt")
    title_fr = ET.SubElement(titleStmt, "title", type ="main")
    title_fr.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"
    title_en = ET.SubElement(titleStmt, "title", type ="main")
    title_en.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "en"

    title_sub_fr = ET.SubElement(titleStmt, "title", type ="sub")
    title_sub_fr.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"
    title_sub_en = ET.SubElement(titleStmt, "title", type ="sub")
    title_sub_en.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "en"         

    # meeting____________________________________________________

        # voir partie Meeting plus bas

    # respStmt________________________________________________________

    personnes = {"Brunel TCHEKELI": "id-hal", "Marie PUREN": "id-hal", "Pierre VERNUS": "id-hal"}

    # Boucle pour créer respStmt pour chaque personne
    for personne, identifiant in personnes.items():
        respStmt = ET.SubElement(titleStmt , "respStmt")
        persName = ET.SubElement(respStmt , "persName")
        forename = ET.SubElement(persName , "forename")
        surname = ET.SubElement(persName , "surname")

        # Définir l'attribut xml:id pour l'élément ptr
        #ptr = ET.SubElement(persName, "ptr")
        #ptr.attrib["type"] = identifiant

         # Définir le texte pour forename et surname
        forename.text = personne.split()[0]  # Utiliser le prénom de la personne
        surname.text = personne.split()[1]  # Utiliser le nom de famille de la personne

        # Vérifier si la personne est "Marie PUREN"
        if personne == "Brunel TCHEKELI":
            # Ajouter l'élément ptr avec les attributs appropriés
            resp_fr = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})                        
            resp_fr.text = "Transformation du JSON en XML-TEI et ajout automatique des balises TEI par des scripts Python"
            resp_en = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
            resp_en.text = "Transformation from JSON to XML-TEI and automatic addition of TEI tags by Python scripts"

        # Vérifier si la personne est "Marie PUREN"
        if personne == "Marie PUREN":
            # Ajouter l'élément ptr avec les attributs appropriés
            ptr = ET.SubElement(persName, "ptr", type= identifiant, target=personne)
            ptr = ET.SubElement(persName, "ptr", type="orcid", target="0000-0001-5452-3913")
            resp_fr = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})                        
            resp_fr.text = "TEI Header"
            resp_en = ET.SubElement(respStmt, "resp", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
            resp_en.text = "TEI Header"

        else:
            # Ajouter l'élément ptr avec les attributs standard
            ptr = ET.SubElement(persName, "ptr", type=identifiant, target=personne)


    # Création de l'élément funder
    funder = ET.SubElement(titleStmt, "funder")
    # Création des éléments orgName avec les attributs xml:lang correspondants
    orgName_fr = ET.SubElement(funder, "orgName", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})
    orgName_fr.text = "Bibliothèque nationale de France"
    orgName_en = ET.SubElement(funder, "orgName", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
    orgName_en.text = "National Library of France"

    # Ajout de extent avec les informations sur le nombre de page, le nombre de mots etc (plus tard)                      

    extent= ET.SubElement(fileDesc, "extent")
    measure_pages_fr = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "fr"})
    measure_pages_fr.text = "pages"
    measure_pages_en = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "en"})
    measure_pages_en.text = "pages"

    measure_utterances_sl = ET.SubElement(extent, "measure", {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "sl"})
    measure_utterances_sl.text = "énoncés"
    measure_utterances_en = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "en"})
    measure_utterances_en.text = "utterances"

    measure_words_sl = ET.SubElement(extent, "measure",  {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "sl"})
    measure_words_sl.text = "mots"
    measure_words_en = ET.SubElement(extent, "measure", {"quantity": "counts", "{http://www.w3.org/XML/1998/namespace}lang": "en"})
    measure_words_en.text = "words"

    global publicationStmt
    publicationStmt = ET.SubElement(fileDesc , "publicationStmt")
    publisher = ET.SubElement(publicationStmt, "publisher")
    publisher.text = "AGODA"
    authority = ET.SubElement(publicationStmt, "authority")
    authority.text = "Bnf Datalab"
    availability = ET.SubElement(publicationStmt, "availability", status="restricted", n="cc-by")
    licence = ET.SubElement(availability, "licence", target="https://creativecommons.org/licenses/by/4.0/")

    # ajout de la date _________________________________

    now = datetime.datetime.now()
    date = ET.SubElement(publicationStmt, "date", {"when": now.strftime("%Y-%m-%d")})
    # date générée automatiquement en utilisant la méthode now() de la classe datetime.datetime et
    # le format est défini avec strftime() en utilisant le modèle "AAAA-MM-JJ"

     # ajout de la dsourceDesc _________________________________
    sourceDesc = ET.SubElement(fileDesc , "sourceDesc")
    biblFull = ET.SubElement(sourceDesc, "biblFull")
    titleStmt_sDc = ET.SubElement(biblFull, "titleStmt")
    title_sDc = ET.SubElement(titleStmt_sDc, "title")
    title_sDc.text = "A définir"
    
    global publicationStmt_sDc
    publicationStmt_sDc = ET.SubElement(biblFull , "publicationStmt")
    publisher_sDc_fr = ET.SubElement(publicationStmt_sDc, "publisher", {"{http://www.w3.org/XML/1998/namespace}lang": "fr"})
    publisher_sDc_fr.text = " "

    publisher_sDc_en = ET.SubElement(publicationStmt_sDc, "publisher", {"{http://www.w3.org/XML/1998/namespace}lang": "en"})
    publisher_sDc_en.text = " "

    pubPlace_sDc = ET.SubElement(publicationStmt_sDc, "pubPlace")
    location_sDc = ET.SubElement(pubPlace_sDc, "location")
    country_sDc = ET.SubElement(location_sDc, "country", key="FR")
    settlement_sDc = ET.SubElement(location_sDc, "settlement", type="city") # que mettre ? est-ce "Paris" pour tout ?
    settlement_sDc.text = " "
    # date____________(voir partie date-pub plus bas: la date est récupérée dans chaque fichier et est ajouté ici)

    distributor_sDc = ET.SubElement(publicationStmt_sDc, "distributor", facs="https://gallica.bnf.fr/ark:/12148/bpt6k477552f/f1")
    distributor_sDc.text = "Source gallica.bnf.fr / Bibliothèque nationale de France"
    availability_sDc = ET.SubElement(publicationStmt_sDc, "availability")
    licence_sDc = ET.SubElement(availability_sDc, "licence", {"target":"https://gallica.bnf.fr/edit/und/conditions-dutilisation-des-contenus-de-gallica", "{http://www.w3.org/XML/1998/namespace}lang": "fr"})
    licence_sDc_p1 = ET.SubElement(licence_sDc, "p" )
    licence_sDc_p1.text = "Les contenus accessibles sur le site Gallica sont pour la plupart des reproductions numériques d'œuvres tombées dans le domaine public provenant des collections de la BnF."
    licence_sDc_p2 = ET.SubElement(licence_sDc, "p" )
    licence_sDc_p2.text = "Ces contenus sont considérés, en vertu du code des relations entre le public et l’administration, comme étant des informations publiques et leur réutilisation s'inscrit dans le cadre des dispositions prévues aux articles L. 321-1 à L. 327-1 de ce code."

    seriesStmt = ET.SubElement(biblFull, "seriesStmt")
    title_series = ET.SubElement(seriesStmt, "title" )
    title_series.text = "Journal Officiel de la République française"
    biblScope1 = ET.SubElement(seriesStmt, "biblScope")
    biblScope1.text = "Débats parlementaires"
    biblScope2 = ET.SubElement(seriesStmt, "biblScope")
    biblScope2.text = "Chambre des députés"
    idno = ET.SubElement(seriesStmt, "idno" , type="ISSN")
    idno.text = "1270-5942"



# EncodingDesc ______________________________________________________________________

    encodingDesc = ET.SubElement(teiHeader , "encodingDesc")

# profileDesc ______________________________________________________________________
    profileDesc = ET.SubElement(teiHeader , "profileDesc")
    langUsage = ET.SubElement(profileDesc, "langUsage")
    language = ET.SubElement(langUsage, "language", ident="fr")
    language.text = "Français"
    global setting_desc
    settingDesc = ET.SubElement(profileDesc, "settingDesc")
    setting_desc = ET.SubElement(settingDesc, "setting")
    name_desc1 = ET.SubElement(setting_desc, "name", type="")    
    name_desc2 = ET.SubElement(setting_desc, "name", type="")
    name_desc3 = ET.SubElement(setting_desc, "name", type="")
   # date_desc = voir partie date plus bas


# Ajout de la description du projet--------------------------------------------------------
    projetDesc = ET.SubElement(encodingDesc, "projetDesc")
    p1 = ET.SubElement(projetDesc, "p")
    p1.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr" # A cause des crochets et trop de "", écrire l'attibut de cette maniène pose moins de problème.
    ref1 = ET.SubElement(p1, "ref", target = "https://www.bnf.fr/fr/les-projets-de-recherche#bnf-agoda")
    ref1.text = "AGODA "           
    ref1.tail = '''est un projet qui a pour objectif de rendre disponible au format XML-TEI les textes de débats parlementaires à la Chambre des députés au cours de la Troisième République, suivant l\''''
    ref2 = ET.SubElement(p1, "ref", target ="https://github.com/mpuren/agoda/blob/ODD/documentation/agoda_odd.xml")
    ref2.text = "ODD "
    ref2.tail = "défini pour le projet à partir des "
    ref3 = ET.SubElement(p1, "ref", target = "https://github.com/clarin-eric/parla-clarin")
    ref3.text =  "recommandations produites par Parla-CLARIN. "
    ref3.tail = ''' Dans une optique de preuve de concept, la phase 1 du projet AGODA se concentre plus particulièrement
    sur la 5ème législature (1889-1893). Les textes encodés sont d'abord extraits des documents numérisés disponibles sur '''
    ref4 = ET.SubElement(p1, "ref", target = "https://gallica.bnf.fr/ark:/12148/cb328020951/date.item")
    ref4.text = "Gallica,"
    ref4.tail = ''' la bibliothèque numérique de la Biliothèque nationale de France, puis ils sont convertis 
    en XML-TEI au moyen de scripts Python.'''

                    # Version anglaise ______________________________________________________________________________

    p2 = ET.SubElement(projetDesc, "p")
    p2.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "en"
    ref1_2 = ET.SubElement(p2, "ref", target = "https://www.bnf.fr/fr/les-projets-de-recherche#bnf-agoda")
    ref1_2.text = "AGODA "
    ref1_2.tail = '''is a project that aims to make available in XML-TEI format the texts of parliamentary debates in the
    Chamber of Deputies during the Third Republic, following the'''
    ref2_2 = ET.SubElement(p2, "ref", target ="https://github.com/mpuren/agoda/blob/ODD/documentation/agoda_odd.xml")
    ref2_2.text = "ODD "
    ref2_2.tail = " defined for the project from the "
    ref3_2 = ET.SubElement(p2, "ref", target = "https://github.com/clarin-eric/parla-clarin")
    ref3_2.text = " Parla-CLARIN recommendations "
    ref3_2.tail = ''' From a proof-of-concept perspective, phase 1 of the AGODA project focuses more specifically on the
    5th legislature (1889-1893). The encoded texts are first extracted from the digitised documents available on '''
    ref4_2 = ET.SubElement(p2, "ref", target = "https://gallica.bnf.fr/ark:/12148/cb328020951/date.item")
    ref4_2.text = "Gallica,"
    ref4_2.tail = '''  the digital library of the Biliothèque nationale de France, then they are converted into
    XML-TEI using Python scripts.'''    

 # fin de la description du projet--------------------------------------------------------    

    pass
  
# Grandes divisions des débats ___________________________________________________________________________

# Création de l'élément racine et le squelette du XML
root = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
teiHeader = ET.SubElement(root, "teiHeader")
text_tei = ET.SubElement(root, "text")
body = ET.SubElement(text_tei, "body")

div_sitting = ET.SubElement(body, "div", attrib={"type": "sitting"})
div_content = ET.SubElement(div_sitting, "div", attrib={"type": "content"})

def grandes_divisions(data, body, fichier_json):
    
    for i in range(len(data)):
        
        if "comment" in data[i]:

            global page_number
            if data[i]['comment'] == 'page-number':
                # Ajouter un élément 'page-number' avec le contenu de la clé 'text_ocr'
                pge_number = ET.SubElement(body, "pb", attrib={"n": "{}".format(page_number)})
                pge_number.text = data[i]['text_ocr']
                #body.insert(0, bp_element)


            elif data[i]['comment'] == 'page-number-ref':
                # Ajouter un élément 'page-number-ref' avec le contenu de la clé 'text_ocr'
                incident = ET.SubElement(body, "incident")
                desc = ET.SubElement(incident, "desc")
                page_number_ref = ET.SubElement(desc, "ref", attrib={"target": "#p"} )
                page_number_ref.text = data[i]['text_ocr'] 

            elif data[i]['comment'] == 'part head':
                # Ajouter un élément 'part head' avec le contenu de la clé 'text_ocr'
                div = ET.SubElement(body, "div", attrib={"type": "part", "corresp": "#pv"})
                part_head = ET.SubElement(div, "head" )
                part_head.text = data[i]['text_ocr'] 
                div.addprevious(etree.Comment(generate_id("Partie_")))


            elif data[i]['comment'] == 'opening seg':
                # Ajouter un élément 'opening seg' avec le contenu de la clé 'text_ocr'
                note = ET.SubElement(body, "note", {"type": "opening", "{http://www.w3.org/XML/1998/namespace}id": "ID_" + filename.split("_")[3] + "_" + generate_id("n")})
                opening_seg = ET.SubElement(note, "seg", {"type": "opening", "{http://www.w3.org/XML/1998/namespace}id": generate_id("s")})
                opening_seg.text = data[i]['text_ocr']  

            elif data[i]['comment'] == 'closing seg':
                # Ajouter un élément 'closing seg' avec le contenu de la clé 'text_ocr'
                note = ET.SubElement(body, "note", {"type": "opening", "{http://www.w3.org/XML/1998/namespace}id": generate_id("note")})
                closing_seg = ET.SubElement(note, "seg", {"type": "opening", "{http://www.w3.org/XML/1998/namespace}id": generate_id("s")} )
                closing_seg.text = data[i]['text_ocr'] 

            elif data[i]['comment'] == 'signed seg back':
                # Ajouter un élément 'signed seg back' avec le contenu de la clé 'text_ocr'
                signed = ET.SubElement(body, "signed")
                signed_seg_back = ET.SubElement(signed, "seg")
                signed_seg_back.text = data[i]['text_ocr']  

            elif data[i]['comment'] == 'signed seg div-end':
                # Ajouter un élément 'signed seg div-end' avec le contenu de la clé 'text_ocr'
                signed = ET.SubElement(body, "signed")
                signed_seg_div_end = ET.SubElement(signed, "seg")
                signed_seg_div_end.text = data[i]['text_ocr']  

            elif data[i]['comment'] == 'text':
                # Ajouter un élément 'text' avec le contenu de la clé 'text_ocr'
                text = ET.SubElement(body)
                text.text = data[i]['text_ocr']  

            elif data[i]['comment'] == 'text-back':
                # Ajouter un élément 'text-back' avec le contenu de la clé 'text_ocr'
                div = ET.SubElement(body, "div")
                back = ET.SubElement(div, "back")
                text_back = ET.SubElement(back, "div" )
                text_back.text = data[i]['text_ocr']  
        pass

    # Paragraphes et divisions correspondants _________________________________________________________________________  

def paragraphes_divisions(data):

    for i in range(len(data)):
        
        if "comment" in data[i]:

            if data[i]['comment'] == 'seg':
                # Ajouter un élément 'seg' avec le contenu de la clé 'text_ocr'
                global seg
                seg = ET.SubElement(body, "seg")
                seg.text = data[i]['text_ocr']


            elif data[i]['comment'] == 'seg-beginning':
                # Ajouter un élément 'seg-beginning' avec le contenu de la clé 'text_ocr'
                seg_beginning = ET.SubElement(seg, "seg")
                seg_beginning.text = data[i]['text_ocr']


            elif data[i]['comment'] == 'seg-end':
                # Ajouter un élément 'seg-end' avec le contenu de la clé 'text_ocr'
               
                seg_end = ET.SubElement(seg, "seg")
                seg_end.text = data[i]['text_ocr']


            elif data[i]['comment'] == 'comment seg':
                # Ajouter un élément 'comment seg' avec le contenu de la clé 'text_ocr'
                note = ET.SubElement(body, "note", attrib={"type": "comment"})
                comment_seg = ET.SubElement(note, "seg")
                comment_seg.text = data[i]['text_ocr']

            elif data[i]['comment'] == 'comment-beginning seg':
                # Ajouter un élément 'comment-beginning seg' avec le contenu de la clé 'text_ocr'
                note = ET.SubElement(body, "note", attrib={"type": "comment"})
                comment_beginning_seg = ET.SubElement(note, "seg")
                comment_beginning_seg.text = data[i]['text_ocr']


            elif data[i]['comment'] == 'comment-end seg':
                # Ajouter un élément 'comment-end seg' avec le contenu de la clé 'text_ocr'
                note = ET.SubElement(body, "note", attrib={"type": "comment"})
                comment_end_seg = ET.SubElement(note, "seg")
                comment_end_seg.text = data[i]['text_ocr']

            elif data[i]['comment'] == 'note seg':
                # Ajouter un élément 'note seg' avec le contenu de la clé 'text_ocr'
                note_seg = ET.SubElement(note, "seg")
                note_seg.text = data[i]['text_ocr']   
        pass

        # Prises de parole_______________________________________________________________________________________________________
    
def prises_de_parole(data, div_content):

    div_sitting = ET.SubElement(body, "div", attrib={"type": "sitting"})
    div_content = ET.SubElement(div_sitting, "div", attrib={"type": "content"})
    u_element = ET.SubElement(div_content, "u")

    for i in range(len(data)):
        
        if "comment" in data[i]:
            
            if data[i]['comment'] == 'u seg':
                
                # Ajouter un élément 'u-seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                u_seg = ET.SubElement(u_element, "seg")
                u_seg.text = data[i]['text_ocr']

            elif data[i]['comment'] == 'u-beginning seg':
                # Ajouter un élément 'u-beginning seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                u_element = ET.SubElement(div_content, "u")
                u_beginning_seg = ET.SubElement(u_element, "seg")
                u_beginning_seg.text = data[i]['text_ocr']

            elif data[i]['comment'] == 'u-end seg':
                # Ajouter un élément 'u-end seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                u_element = ET.SubElement(div_content, "u")
                u_end_seg = ET.SubElement(u_element, "seg")
                u_end_seg.text = data[i]['text_ocr']
        pass

    # Cas particuliers ______________________________________________________________________________________________________

def cas_particuliers(data):
    
        for i in range(len(data)):
            
            if "comment" in data[i]:
                
                if data[i]['comment'] == 'quote seg':
                    # Ajouter un élément 'quote seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                    global seg
                    seg = ET.SubElement(body, "seg")
                    quote_seg = ET.SubElement(seg, "quote")
                    quote_seg.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'quote-beginning seg':
                    # Ajouter un élément 'quote-beginning seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                    quote_beg_seg = ET.SubElement(body, "quote")
                    quote_beg_seg.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'quote-end seg':
                    # Ajouter un élément 'quote-end seg' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                    quote_end_seg = ET.SubElement(body, "quote")
                    quote_end_seg.text = data[i]['text_ocr']          

                elif data[i]['comment'] == 'incident':
                    # Ajouter un élément 'incident' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                    u_element = ET.SubElement(body, "u")
                    incident = ET.SubElement(u_element, "seg")
                    incident.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'incident-beginning':
                    # Ajouter un élément 'incident-beginning' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                    incident_beginning = ET.SubElement(incident, "incident-beginning")            
                    incident_beginning.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'incident-end':
                    # Ajouter un élément 'incident-end' avec un élément 'seg' contenant le contenu de la clé 'text_ocr'
                    incident_end = ET.SubElement(incident, "incident-end")            
                    incident_end.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'table':
                    # Ajouter un élément 'table' avec le contenu de la clé 'text_ocr'
                    table = ET.SubElement(body, "table")
                    row = ET.SubElement(table, "row")
                    cell = ET.SubElement(row, "cell")
                    cell.text = data[i]['text_ocr'] 

                elif data[i]['comment'] == 'other-sitting':
                    # Ajouter un élément 'other-sitting' avec le contenu de la clé 'text_ocr'
                    div = ET.SubElement(body, "div")
                    div1 = ET.SubElement(div, "div", attrib={"type": "other-sitting"})
                    head = ET.SubElement(div1, "head")
                    head.text = data[i]['text_ocr']
            pass

    # Éléments spécifiques à la première page________________________________________________________________________________

def specifics_elements(data):
    
    global publicationStmt_sDc

    list_item = ET.SubElement(div_content, "list")

    for i in range(len(data)):
        if "comment" in data[i]:
            if data[i]['comment'] == "sitting contents":
                # Ajouter un élément 'sitting contents' avec le contenu de la clé 'text_ocr'

                if data[i]['text_ocr'] == "SOMMAIRE" or data[i]['text_ocr'] == "Sommaire":
                    sc_element = ET.SubElement(div_content, "head")
                    sc_element.text = data[i]['text_ocr']
                    div_content.insert(0, sc_element)
                else :
                    sc_element.text = data[i]['text_ocr']
                    div_content.insert(0, sc_element)

            elif data[i]['comment'] == 'body page-number date-pub':
                # Ajouter un élément 'body page-number date-pub' avec le contenu de la clé 'text_ocr'
                global bp_element
                bp_element = ET.SubElement(body, "pb", attrib={"n": "{}".format(page_number)})
                body.insert(0, bp_element)

            # récupération de la date de publication ___________________________________________________

                texte_date = data[i]['text_ocr'] 

                partie_date = re.search(r'\d{1,2} [a-zA-Z]+ \d{4}', texte_date).group()

                # fonction pour la reconnaissance des dates. 

                parsed_date = dateparser.parse(partie_date, languages=['fr'])
                # Formatter la date 
                formatted_date = parsed_date.strftime('%Y-%m-%d')

                # Création de l'élément date avec l'attribut when
                global publicationStmt_sDc
                global setting_desc
                date_sDc = ET.SubElement(publicationStmt_sDc, "date", {"when": formatted_date})
                date_sDc.text = partie_date
                date_desc = ET.SubElement(setting_desc, "date", {"when": formatted_date})
                date_desc.text = partie_date

            elif data[i]['comment'] == 'item':
                # Ajouter un élément 'item' avec le contenu de la clé 'text_ocr'

                item = ET.SubElement(list_item, "item")
                item.text = data[i]['text_ocr']


            elif data[i]['comment'] == 'item-list':
                # Ajouter un élément 'item-list' avec le contenu de la clé 'text_ocr'

                item_list = ET.SubElement(list_item, "item")
                item_list.text = data[i]['text_ocr']
                div_content.insert(-1, sc_element)

            elif data[i]['comment'] == "meeting-session meeting-legislature useless":
                # Ajouter un élément 'meeting-session meeting-legislature' avec le contenu de la clé 'text_ocr'
                meet_session = data[i]['text_ocr']

                # extrait le numéro de session
                num_session = int(meet_session.split(' ')[-1][:-4])
                num_legis = int(meet_session.split(' ')[0][:-1])

                # extrait le texte de la session
                texte_session = "Session " + meet_session.split('Session ')[1]  # ajoute le mot "Sessio
                # extrait la première lettre du mot suivant "Session"
                lettre_session = texte_session.split('Session ')[1][0].upper()

                # créer la balise
                global publicationStmt
                meeting_session = ET.SubElement(publicationStmt, "meeting", n=f"E{num_session}", ana="#parla.lower\n#parla.session") # une autre manière d'écrire les attributs
                meeting_session.text = f"{texte_session}"

                meet_legis = meet_session.split('—')
                meeting_legislature = ET.SubElement(publicationStmt, "meeting", n=f"{num_legis}                                                    L", ana="#parla.lower\n#parla.legislature")
                meeting_legislature.text = meet_legis[0]
                
                global titleStmt
                meeting_session_t = ET.SubElement(titleStmt, "meeting", n=f"E{num_session}", ana="#parla.lower\n#parla.session") # une autre manière d'écrire les attributs
                meeting_session_t.text = f"{texte_session}".strip(".")
                titleStmt.insert(4, meeting_session_t)

                meet_legis = meet_session.split('—')
                meeting_legislature_t = ET.SubElement(titleStmt, "meeting", n=f"{num_legis}L", ana="#parla.lower\n#parla.legislature")
                meeting_legislature_t.text = meet_legis[0].split(".")[0].replace("°", "e")
                titleStmt.insert(5, meeting_legislature_t)

            elif data[i]['comment'] == "meeting-sitting useless" or data[i]['comment'] == "meeting-sitting":
                # Ajouter un élément 'meeting-sitting' avec le contenu de la clé 'text_ocr'
                meet_sit = data[i]['text_ocr'].split('—')
                num_seance = meet_sit[1][0:3]
                meeting_sitting = ET.SubElement(publicationStmt, "meeting", n=f"{num_seance}", ana="#parla.lower\n#parla.sitting")
                meeting_sitting.text = data[i]['text_ocr']
                meeting_sitting_t = ET.SubElement(titleStmt, "meeting", n=f"{num_seance}", ana="#parla.lower\n#parla.sitting")
                meeting_sitting_t.text = data[i]['text_ocr'].split("—")[1].strip(".").replace("°", "e").lower()
                titleStmt.insert(6, meeting_sitting_t)

            elif data[i]['comment'] == 'agenda':
                # Ajouter un élément 'agenda' avec le contenu de la clé 'text_ocr'
                div_agenda = ET.SubElement(body,"div")
                agenda_head = ET.SubElement(div_agenda, "head", type="agenda")
                agenda_head.text = data[i]['text_ocr']
        pass

    # Annexes________________________________________________________________________________________________________________

def annexes(data):
    
        for i in range(len(data)):
            if "comment" in data[i]:
                
                if data[i]['comment'] == 'part1':
                # Ajouter un élément 'part1' avec le contenu de la clé 'text_ocr'
                    div_annexe = ET.SubElement(body, "div", attrib={"type": "part"})
                    div_annexe.text = data[i]['text_ocr']
                    div_annexe.addprevious(etree.Comment(generate_id("Partie_")))

                elif data[i]['comment'] == 'part1 u-beginning seg':
                    # Ajouter un élément 'part1 u-beginning seg' avec le contenu de la clé 'text_ocr'
                    div_annexe = ET.SubElement(body, "div", attrib={"type": "part"})
                    part_u_beg = ET.SubElement(div_annexe, "u")
                    seg_part1 = ET.SubElement(part_u_beg, "seg")
                    seg_part1.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'erratum':
                    # Ajouter un élément 'erratum' avec le contenu de la clé 'text_ocr'
                    div_erratum = ET.SubElement(body, "div", attrib={"type": "erratum"})
                    head_annexe = ET.SubElement(div_erratum, "head")
                    label_annexe = ET.SubElement(head_annexe, "label")
                    label_annexe.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'note-head':
                    # Ajouter un élément 'note-head' avec le contenu de la clé 'text_ocr'
                    div_erratum = ET.SubElement(body, "div", attrib={"type": "erratum"})
                    head_annexe = ET.SubElement(div_erratum, "head")
                    note_head_annexe = ET.SubElement(head_annexe,"note")
                    note_head_annexe.text = data[i]['text_ocr']


                elif data[i]['comment'] == 'lists':
                    # Ajouter un élément 'lists' avec le contenu de la clé 'text_ocr'
                    div_lists = ET.SubElement(body, "div", attrib={"type": "lists"})
                    head_lists = ET.SubElement(div_lists, "head")
                    label_lists = ET.SubElement(head_lists, "label")
                    label_lists.text = data[i]['text_ocr']


                elif data[i]['comment'] == 'offices': 
                    # Ajouter un élément 'offices' avec le contenu de la clé 'text_ocr'
                    div_offices = ET.SubElement(body, "div", attrib={"type": "offices"})
                    head_offices = ET.SubElement(div_offices, "head")
                    head_offices.text = data[i]['text_ocr']     


                elif data[i]['comment'] == 'appendices': 
                    # Ajouter un élément 'appendices' avec le contenu de la clé 'text_ocr'
                    div_appendices = ET.SubElement(body, "div", attrib={"type": "appendices"})
                    head_appendices = ET.SubElement(div_appendices, "head")
                    head_appendices.text = data[i]['text_ocr']


                elif data[i]['comment'] == 'voting1' or data[i]['comment'] == 'voting1': 
                    # Ajouter un élément 'voting1' avec le contenu de la clé 'text_ocr'
                    global div_voting1
                    div_voting1 = ET.SubElement(body, "div", attrib={"type": "voting"})
                    head_voting1 = ET.SubElement(div_voting1, "head")
                    label_voting1 = ET.SubElement(head_voting1, "label")
                    label_voting1.text = data[i]['text_ocr']


                elif data[i]['comment'] == 'result':
                    # Ajouter un élément 'result' avec le contenu de la clé 'text_ocr'
                    note_result = ET.SubElement(div_voting1, "note", attrib={"type": "result"})
                    seg_note_annexe = ET.SubElement(note_result, "seg")
                    seg_note_annexe.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'voterslist-beginning': 
                    # Ajouter un élément 'voterslist-beginning' avec le contenu de la clé 'text_ocr'
                    global note_voterlist
                    note_voterlist = ET.SubElement(div_voting1, "note", attrib={"type": "voterlist"})
                    voterlist = ET.SubElement(note_voterlist, "desc")
                    voterlist.text = data[i]['text_ocr']


                elif data[i]['comment'] == 'desc':  
                    # Ajouter un élément 'desc' avec le contenu de la clé 'text_ocr'
                    voterlist_desc = ET.SubElement(note_voterlist, "desc")
                    voterlist_desc.text = data[i]['text_ocr']


                elif data[i]['comment'] == 'seg note-beginning': 
                    # Ajouter un élément 'seg note-beginning' avec le contenu de la clé 'text_ocr'
                    note_note_beg = ET.SubElement(div_voting1, "note", attrib={"type": "numbersannounced"})
                    seg_note_beg = ET.SubElement(note_note_beg, "seg")
                    seg_note_beg.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'seg note-end' or data[i]['comment'] == 'seg note-end div-end': 
                    # Ajouter un élément 'seg note-end' avec le contenu de la clé 'text_ocr'
                    seg_note_end = ET.SubElement(div_voting1, "seg")
                    seg_note_end.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'rectification':
                    # Ajouter un élément 'rectification' avec le contenu de la clé 'text_ocr'
                    div_rectification = ET.SubElement(body, "div", attrib={"type": "rectification"})
                    head_rectification = ET.SubElement(div_rectification, "head")
                    head_rectification.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'seg comment-beginning': 
                    # Ajouter un élément 'seg comment-beginning' avec le contenu de la clé 'text_ocr'
                    note_com_beg = ET.SubElement(body, "note", attrib={"type": "comment"})
                    seg_com_beg = ET.SubElement(note_com_beg, "seg")
                    seg_com_beg.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'seg note-end':  
                    # Ajouter un élément 'seg note-end' avec le contenu de la clé 'text_ocr'
                    seg_com_end = ET.SubElement(note_com_beg, "seg")
                    seg_com_end.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'petition':
                    # Ajouter un élément 'petition' avec le contenu de la clé 'text_ocr'
                    div_petition = ET.SubElement(body, "div", attrib={"type": "petition"})
                    head_petition = ET.SubElement(div_petition, "head")
                    label_petition = ET.SubElement(head_petition, "label")
                    label_petition.text = data[i]['text_ocr']

                elif data[i]['comment'] == 'note-head':   
                    # Ajouter un élément 'note-head' avec le contenu de la clé 'text_ocr'
                    note_petition = ET.SubElement(head_petition, "note")
                    note_petition.text = data[i]['text_ocr']
            pass

def baliseur_data(data, body_element, filename ):
    grandes_divisions(data, body_element, filename)
    paragraphes_divisions(data)
    prises_de_parole(data, body_element)
    cas_particuliers(data)
    annexes(data)
    pass
    
def baliseur_teiheader(data, body_element, filename):
    create_teiheader(data)
    specifics_elements(data)
    pass    

# Définir la fonction pour écrire un commentaire _________________________________________________

def write_comment(filename):
    comment_text = f"New page added from {filename}"
    comment_element = ET.Comment(comment_text)
    body.append(comment_element)
    return comment_element

# Chemin absolu du dossier contenant les fichiers à parcourir
dossier_json = os.path.join(os.getcwd(), "json_data")
dossier_xml = os.path.join(os.getcwd(), "xml_data")

# Créer un pattern regex pour extraire le numéro de page du nom de fichier
page_number_pattern = re.compile(r'^.*_p(\d+)\.json$')

# Récupérer la liste des fichiers JSON dans le dossier
fichiers = [f for f in os.listdir(dossier_json) if f.endswith('.json')]

# Trier les fichiers JSON en fonction du numéro de page
fichiers_json_tries = sorted(fichiers, key=lambda x: int(page_number_pattern.match(x).group(1)))

# Parcourir les fichiers JSON triés
for index, fichier_json in enumerate(fichiers_json_tries):

    fichier_json = os.path.join(dossier_json, fichier_json)
    root.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"
    filename = os.path.basename(fichier_json)
    xml_id = os.path.splitext(filename)[0]
    root.attrib["{http://www.w3.org/XML/1998/namespace}id"] = xml_id
    global page_number
    page_number = filename.split("_p0")[1].split(".")[0]
    
    with open(fichier_json, "r", encoding="utf-8") as f:
        # Lire le contenu du fichier JSON
        data = json.load(f)
        
        write_comment(filename)
        baliseur_data(data, body, fichier_json)
    
        if index == 0:
            create_teiheader(data)
            specifics_elements(data)
        
         # Création du fichier XML
xml_tree = ET.ElementTree(root)
final_output = filename.split("-")[0] + "_compiled" + ".xml"
xml_filename = os.path.join(dossier_xml, final_output )
        
#   # Création du fichier XML
# xml_tree = ET.ElementTree(root)
# output = filename.split(".")[0] + ".xml"
# xml_filename = os.path.join(dossier_xml, output )

with open(os.path.join(dossier_xml, xml_filename), "wb") as xml_file:
     root = xml_tree.getroot()

     instruction1 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://purl.oclc.org/dsdl/schematron"'
     model_instr1 = ET.ProcessingInstruction("xml-model", instruction1)
     root.addprevious(model_instr1)

     instruction2 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"'
     model_instr2 = ET.ProcessingInstruction("xml-model", instruction2)
     root.addprevious(model_instr2)

     xml_tree.write(xml_file, encoding="utf-8", xml_declaration=True, pretty_print=True)
    
# la partie ci dessous du code n'est pas obligatoire. le code sert juste à enregistrer chaque fichier xml séparémment______
    
    
# Chemin absolu du dossier contenant les fichiers à parcourir
dossier_json = os.path.join(os.getcwd(), "json_data")
dossier_xml = os.path.join(os.getcwd(), "xml_data")

# Créer un pattern regex pour extraire le numéro de page du nom de fichier
page_number_pattern = re.compile(r'^.*_p(\d+)\.json$')

# Récupérer la liste des fichiers JSON dans le dossier
fichiers = [f for f in os.listdir(dossier_json) if f.endswith('.json')]

# Trier les fichiers JSON en fonction du numéro de page
fichiers_json_tries = sorted(fichiers, key=lambda x: int(page_number_pattern.match(x).group(1)))

# Parcourir les fichiers JSON triés
for index, fichier_json in enumerate(fichiers_json_tries):
    fichier_json = os.path.join(dossier_json, fichier_json)
    with open(fichier_json, "r", encoding="utf-8") as f:
        # Lire le contenu du fichier JSON
        data = json.load(f)

        # Création de l'élément racine et le squelette du XML
        root = ET.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
        root.attrib["{http://www.w3.org/XML/1998/namespace}lang"] = "fr"

        filename = os.path.basename(fichier_json)
        xml_id = os.path.splitext(filename)[0]
        root.attrib["{http://www.w3.org/XML/1998/namespace}id"] = xml_id

        teiHeader = ET.SubElement(root, "teiHeader")
        text_tei = ET.SubElement(root, "text")
        body = ET.SubElement(text_tei, "body")
        global page_number
        page_number = filename.split("_p0")[1].split(".")[0]

        # Ajouter le commentaire pour les fichiers suivants
        if index > 0:
            write_comment(filename)

        if index == 0:
            create_teiheader(data)
        specifics_elements(data)
        baliseur_data(data, body, fichier_json)

        # Création du fichier XML
        xml_tree = ET.ElementTree(root)
        output = xml_id + ".xml"
        xml_filename = os.path.join(dossier_xml, output)

        with open(xml_filename, "wb") as xml_file:
            instruction1 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://purl.oclc.org/dsdl/schematron"'
            model_instr1 = ET.ProcessingInstruction("xml-model", instruction1)
            root.addprevious(model_instr1)

            instruction2 = ' href="agoda_schema.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"'
            model_instr2 = ET.ProcessingInstruction("xml-model", instruction2)
            root.addprevious(model_instr2)

            xml_tree.write(xml_file, encoding="utf-8", xml_declaration=True, pretty_print=True)


In [2]:
create_teiheader
grandes_divisions
paragraphes_divisions
cas_particuliers
specifics_elements
annexes

<function __main__.annexes(data)>